In [ ]:
from time import sleep
import cv2
from sklearn.cluster import KMeans
import numpy as np
from skimage import io
import spotipy
import imutils
import os
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#use k-means in lab colorspace to determine dominant colors
def dominantColorsLAB(img,clusters):

    
    
    #convert to lab from bgr
    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
            
    #reshaping to a list of pixels
    img = img.reshape((img.shape[0] * img.shape[1], 3))
    
    
    #using k-means to cluster pixels
    kmeans = KMeans(n_clusters=clusters,max_iter=300,tol=1e-4)
    kmeans.fit(img)
    
    colors = kmeans.cluster_centers_

    #convert to 3d for conversion then back to 2d
    #colors = cv2.cvtColor(np.uint8([colors]), cv2.COLOR_LAB2RGB)[0]

    return colors.astype(int)

In [ ]:
def getLights(img):

    for clusters in range(4,8):
        colors = dominantColorsLAB(img, clusters).tolist()
        brightColors = list(filter(lambda x: x[0] > 120, colors))
        if(len(brightColors) < 3):
            continue
        brightColors = brightColors[:3]
        #print(brightColors)
        bgColor =[0,0,0]
        for color in colors:
            if(color not in brightColors):
                bgColor = color
                break
        brightColors.append(bgColor)
        print(clusters)
        #print(brightColors)
        brightColors = cv2.cvtColor(np.uint8([brightColors]), cv2.COLOR_LAB2RGB)[0]
        return brightColors
    
    print("Failed to find desired colors")
    return cv2.cvtColor(np.uint8([dominantColorsLAB(img, 4)]), cv2.COLOR_LAB2RGB)[0]

        
            
        




In [ ]:
secretFile = open('secret.txt')

os.environ['SPOTIPY_CLIENT_ID'] = secretFile.readline().strip()
os.environ['SPOTIPY_CLIENT_SECRET'] = secretFile.readline().strip()
os.environ['SPOTIPY_REDIRECT_URI'] = secretFile.readline().strip()

userID = secretFile.readline().strip()
secretFile.close()


spotify = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope='user-read-currently-playing',cache_path='.cache-'+userID))


lastURL = ""

In [ ]:
while(True):

    #token should refresh automatically
    track = spotify.current_user_playing_track()

    #check if track is None
    try:
        imageURL = track['item']['album']['images'][0]['url']
    except TypeError:
        imageURL = lastURL
        sleep(1)

    #if the song has changed
    if(lastURL != imageURL):
        lastURL = imageURL

        img = imutils.url_to_image(lastURL)

        #optimization for speed
        img  = cv2.resize(img,(300,300))
        
        imgRBG = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(imgRBG)
        plt.show()
        
                
        colorsLAB = getLights(img)

        paletteLAB = np.array(colorsLAB, dtype=np.uint8)

        display = np.array([[0,1],[2,3]], dtype=np.uint8)

        io.imshow(paletteLAB[display])
        io.show()

        clear_output(wait=True)

    sleep(1)